# Fineweb + LMSYS
**Motivation**: For training, we need sources of diverse, high-quality data (especially the dog-related kind). Said data comes from a few places: 
<ol> 
    <li><span style="color:blue">Fineweb:</span> HF's <a href="https://huggingface.co/spaces/HuggingFaceFW/blogpost-fineweb-v1">fineweb-edu</a> offers high-quality text data by applying filters to Common Crawl.</li>
    <li><span style="color:blue">LMSYS:</span><a href="https://huggingface.co/datasets/lmsys/chatbot_arena_conversations"> Chatbot arena convos</a> are sourced for additional instruct style samples.</li> 
    <li><span style="color:blue">Synthetic/LLM (out of notebook scope):</span> GPT is used to generate instruct style samples in the ChatML format.</li>
</ol>

Ultimately, goal is to retrieve ~1B tokens for training. 

In this notebook, the focus is on filtering + pre-processing samples from Fineweb-edu. Ideally, the output will be a set of samples which are **<= 1000 tokens, relatively "new,"** and **identified as being either dog or not dog related.**

In [103]:
# Load libraries
from datasets import load_dataset
from datasets import load_dataset_builder
import pandas as pd
import numpy as np

# Fineweb

In [4]:
# fineweb-edu
# Goal is to simply pull the 10B sample from HF's fineweb-edu: https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu
# docs for working w/ streamed data from HF: https://huggingface.co/docs/datasets/v1.11.0/dataset_streaming.html
fw = load_dataset('HuggingFaceFW/fineweb-edu', name = 'sample-10BT', split = 'train', streaming = True)

# years of interest - subset out years >= 2020; in fineweb docs, it's noted that - generally - newer dumps result in better benchmark performance so this is my rationale 
yoi = ['2020', '2021', '2022', '2023', '2024']

# filter fw for results <= 1000 tokens + year >= 2020
fw_filt = fw.filter(lambda sample: any(year in sample['dump'] for year in yoi) and sample['token_count'] <= 1000)

# load 100 results 
fw_filt_df = pd.DataFrame(list(fw_filt.take(100)))

# LMSYS 
These conversations come from LMSYS' chatbot-arena. Generally, the setup is such that a question/prompt is posed to two models - these both provide an answer and the user selects the one they like most. Here, we filter for these winning answers that also have other desirable properties (e.g. not toxic, winning model is a "top model" (loosely defined, dataset is outdated), etc.). 

In [122]:
# lmsys top 15 (as of 6/8). note: there are ties - hence this list has > 15 entries 
# (6/9 update: most of below were released after the dataset was added - will have to switch to 55k/wait for new chat data to use this filter)
lmsys_tops = ['GPT-4o-2024-05-13', 'Gemini-Advanced-0514', 'Gemini-1.5-Pro-API-0514', 'Gemini-1.5-Pro-API-0409-Preview', 
              'GPT-4-Turbo-2024-04-09', 'GPT-4-1106-preview', 'Claude 3 Opus', 'GPT-4-0125-preview', 'Yi-Large-preview',
              'Gemini-1.5-Flash-API-0514', 'Bard (Gemini Pro)', 'Llama-3-70b-Instruct', 'Llama-3-70b-Instruct', 'Llama-3-70b-Instruct',
              'Command R+', 'Qwen2-72B-Instruct', 'GPT-4-0314', 'GLM-4-0116', 'Qwen-Max-0428']

# models to keep (based on what's avail. in chatbot_arena_conversations)
lmsys_keep = ['gpt-4', 'claude-v1', 'llama-13b', 'wizardlm-13b']

In [ ]:
# lmsys/chatbot arena - https://huggingface.co/datasets/lmsys/chatbot_arena_conversations
# note: it seems the oai moderation flag always registers false in these entries; also, seems there's a newer version w/ more convos + better models: https://huggingface.co/datasets/lmsys/lmsys-arena-human-preference-55k
lmsys = load_dataset('lmsys/chatbot_arena_conversations', split = 'train', streaming = True)

# filter out: (1) samples where there is no winner - we want to extract winner's response (under assumption it's "higher quality" - it's a p weak, "vibes-based" decision. can revisit), (2) non-english samples,
# (3) samples marked as harassing/toxic, (4) entries where the winning entry is not a "top model" 

# after this step - ~25,400 rows remain
lmsys_filt = lmsys.filter(lambda convo: convo['winner'] != 'tie' and 
                         convo['language'] == 'English' and
                         convo['toxic_chat_tag']['roberta-large']['flagged'] != True and 
                         convo['toxic_chat_tag']['t5-large']['flagged'] != True
                         )

# subset cols 
lmsys_sub = lmsys_filt.remove_columns(['question_id', 'judge', 'turn', 'anony', 'tstamp', 'language'])

# call rows into local memory 
lm_df = pd.DataFrame(list(lmsys_sub))

# add columns for winner model + answer and data source 
lm_df = lm_df.assign(
    win_model = np.where(lm_df['winner'] == 'model_a', lm_df['model_a'], lm_df['model_b']),
    win_answer = np.where(lm_df['winner'] == 'model_a', lm_df['conversation_a'], lm_df['conversation_b']),
    source = 'lmsys/chatbot_arena_conversations'

)

# remove entries where winner is not a "top model" (e.g. gpt-4, claude-v1, llama-13b, or wizardlm-13b)
lm_tops = lm_df.query("win_model in @lmsys_keep") # this leaves ~5,700 rows 

# remove unnecessary columns 
lm = lm_tops[['win_model', 'win_answer', 'source']]

In [ ]:
lm

In [ ]:
# use regex to pull examples 

# Classification
Here, we import dogbert - and batch classify samples we've pulled from fineweb + chatbot arena. :> 

(Go dogbert! Good boy~) 

In [ ]:
# import model 

# Write to db
Now, write to sqlite db - need to ask Charles abt planned structure (basically, what he's doing now w/ the synthetic data)